In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from src.preprocessing import preprocess_data
from src.optimization import calculate_optimization_inputs, optimize_portfolio, plot_efficient_frontier_and_portfolios

# Style setup
plt.style.use('seaborn-v0_8-whitegrid')

In [ ]:
MODEL_PATH_ARIMA = 'models/arima_model.pkl'
best_model = joblib.load(MODEL_PATH_ARIMA)

RAW_DATA_PATH = 'data/raw/financial_data.csv'
raw_data = pd.read_csv(RAW_DATA_PATH, header=[0, 1], index_col=0, parse_dates=True)
adj_close_prices = preprocess_data(raw_data)

In [ ]:
print("Retraining ARIMA model on the full dataset...")
best_model.fit(adj_close_prices['TSLA'])

# Forecast 12 months into the future (approx. 252 trading days)
future_forecast_days = 252
future_forecast, future_conf_int = best_model.predict(n_periods=future_forecast_days, return_conf_int=True)

# Create a future date range
future_dates = pd.date_range(start=adj_close_prices.index[-1] + pd.Timedelta(days=1), periods=future_forecast_days, freq='B')
future_forecast = pd.Series(future_forecast, index=future_dates)

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(adj_close_prices['TSLA'][-500:], label='Historical Price') # Plot last ~2 years
plt.plot(future_forecast, label='12-Month Forecast', color='green', linestyle='--')
plt.fill_between(future_dates, 
                 future_conf_int[:, 0], 
                 future_conf_int[:, 1], 
                 color='k', alpha=0.1, label='95% Confidence Interval')
plt.title('TSLA 12-Month Price Forecast')
plt.legend()
plt.show()